In [57]:
#1. Read your Titanic dataset as usual: A training set and Testing set Apply decision tree. 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.read_csv("titanic-passengers.csv", sep=';')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,343,No,2,"Collander, Mr. Erik Gustaf",male,28.0,0,0,248740,13.0000,NaN,S
1,76,No,3,"Moen, Mr. Sigurd Hansen",male,25.0,0,0,348123,7.6500,F G73,S
2,641,No,3,"Jensen, Mr. Hans Peder",male,20.0,0,0,350050,7.8542,NaN,S
3,568,No,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.0750,NaN,S
4,672,No,1,"Davidson, Mr. Thornton",male,31.0,1,0,F.C. 12750,52.0000,B71,S


In [58]:
df.Age = df.Age.fillna(df.Age.median())
num_survived = {'Survived':{'No':0, 'Yes':1}}
df.replace(num_survived, inplace = True)
num_sex = {'Sex':{'male':0, 'female':1}}
df.replace(num_sex, inplace = True)    
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,343,0,2,"Collander, Mr. Erik Gustaf",0,28.0,0,0,248740,13.0000,NaN,S
1,76,0,3,"Moen, Mr. Sigurd Hansen",0,25.0,0,0,348123,7.6500,F G73,S
2,641,0,3,"Jensen, Mr. Hans Peder",0,20.0,0,0,350050,7.8542,NaN,S
3,568,0,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",1,29.0,0,4,349909,21.0750,NaN,S
4,672,0,1,"Davidson, Mr. Thornton",0,31.0,1,0,F.C. 12750,52.0000,B71,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,1,0,237736,30.0708,NaN,C
887,61,0,3,"Sirayanian, Mr. Orsen",0,22.0,0,0,2669,7.2292,NaN,C
888,535,0,3,"Cacic, Miss. Marija",1,30.0,0,0,315084,8.6625,NaN,S
889,102,0,3,"Petroff, Mr. Pastcho (""Pentcho"")",0,28.0,0,0,349215,7.8958,NaN,S


In [59]:
df.drop('Cabin', axis=1, inplace=True)
df['Embarked'].fillna('S', inplace = True)
df.drop('PassengerId', axis=1, inplace=True)
df['Title']=0
df['Title']=df.Name.str.extract('([A-Za-z]+)\.')
df['Title'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col',
                         'Rev','Capt','Sir','Don'],['Miss','Miss','Mrs','Officer','Officer','Royalty','Royalty','Royalty','Officer','Officer','Officer','Royalty','Royalty'],inplace=True)
df.drop('Name', axis=1, inplace=True)
Embarked_one_hot = pd.get_dummies(df['Embarked'])
df = df.join(Embarked_one_hot)
df.drop('Embarked', axis=1, inplace=True)
df.drop('Ticket', axis=1, inplace=True)
for dataset in [df]:
    dataset['Familysize'] = dataset['SibSp'] + dataset['Parch'] +1
df.drop('SibSp', axis=1, inplace=True)
df.drop('Parch', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    int64  
 3   Age         891 non-null    float64
 4   Fare        891 non-null    float64
 5   Title       891 non-null    object 
 6   C           891 non-null    uint8  
 7   Q           891 non-null    uint8  
 8   S           891 non-null    uint8  
 9   Familysize  891 non-null    int64  
dtypes: float64(2), int64(4), object(1), uint8(3)
memory usage: 51.5+ KB


In [60]:
df.isnull().sum().sum()

0

In [63]:
df

,Survived,Pclass,Sex,Age,Fare,Title,C,Q,S,Familysize
0,0,2,0,28.0,13.0000,Mr,0,0,1,1
1,0,3,0,25.0,7.6500,Mr,0,0,1,1
2,0,3,0,20.0,7.8542,Mr,0,0,1,1
3,0,3,1,29.0,21.0750,Mrs,0,0,1,5
4,0,1,0,31.0,52.0000,Mr,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...
886,1,2,1,14.0,30.0708,Mrs,1,0,0,2
887,0,3,0,22.0,7.2292,Mr,1,0,0,1
888,0,3,1,30.0,8.6625,Miss,0,0,1,1
889,0,3,0,28.0,7.8958,Mr,0,0,1,1


In [67]:
from sklearn.model_selection import train_test_split
from sklearn import tree   
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn import metrics 
x = df(['Pclass', 'Sex', 'Age', 'Fare', 'C', 'Q', 'S', 'Familysize', 'Title'], axis=1)

y = df['Survived']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20,random_state=10)

tree = tree.DecisionTreeClassifier()  
tree.fit(x_train, y_train)   
y_pred=tree.predict(x_test)   
print("score:{}".format(accuracy_score(y_test, y_pred)))

TypeError: 'DataFrame' object is not callable